In [ ]:
import os
import time
import math
import random
import csv
from collections import Counter

import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from scipy.spatial.distance import directed_hausdorff
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix, eye
from scipy.sparse.linalg import inv
from sklearn.metrics import f1_score, accuracy_score

In [ ]:
DATA_PATH        = '/home/dime/Desktop/Thesis/pneumoniamnist_224'
FEAT_TRAIN_FILE  = 'vgg_pneumonia_train_features.pth'
FEAT_VAL_FILE    = 'vgg_pneumonia_val_features.pth'
FEAT_TEST_FILE   = 'vgg_pneumonia_test_features.pth'


LABEL_TRAIN_FILE = 'train_labels.npy'
VAL_LABEL_FILE   = 'val_labels.npy'
TEST_LABEL_FILE  = 'test_labels.npy'

CORESET_PREF     = 'coreset_indices_pneumonia_vgg_'

K                = 10
PROP_ALPHA       = 0.5
PERCENTAGES      = [0.1, 1, 10]
IMG_SIZE         = 224
BATCH            = 16
EPOCHS           = 30
LR               = 1e-4
LAPLACE_LAMBDA   = 1e-3
EPS              = 1e-6
PATIENCE         = 5

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([.485, .456, .406], [.229, .224, .225]),
])
val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([.485, .456, .406], [.229, .224, .225]),
])

class NumpyDataset(Dataset):
    def __init__(self, images_path, labels_path, transform=None):
        self.images = np.load(images_path)
        self.labels = np.load(labels_path).flatten()
        self.transform = transform
    def __len__(self):
        return len(self.images)
    def __getitem__(self, i):
        img = self.images[i]
        if img.ndim == 2:
            img = np.stack([img] * 3, axis=-1)
        img = (img * 255).astype(np.uint8)
        img = Image.fromarray(img)
        if self.transform:
            img = self.transform(img)
        return img, int(self.labels[i])

class CoresetDataset(Dataset):
    def __init__(self, base_ds, indices, soft_labels):
        self.base = base_ds
        self.indices = np.array(indices, dtype=int)
        self.y_soft = torch.from_numpy(soft_labels[self.indices]).float()
    def __len__(self):
        return len(self.indices)
    def __getitem__(self, i):
        img, _ = self.base[self.indices[i]]
        return img, self.y_soft[i], self.indices[i]

class SoftDataset(Dataset):
    def __init__(self, base_ds, true_labels, num_classes):
        self.base = base_ds
        Y = np.eye(num_classes, dtype=np.float32)[true_labels]
        self.Y = torch.from_numpy(Y)
    def __len__(self):
        return len(self.base)
    def __getitem__(self, i):
        img, _ = self.base[i]
        return img, self.Y[i], i
    
train_labels = np.load(os.path.join(DATA_PATH, 'train_labels.npy')).flatten()
val_labels   = np.load(os.path.join(DATA_PATH, VAL_LABEL_FILE)).flatten()
test_labels  = np.load(os.path.join(DATA_PATH, TEST_LABEL_FILE)).flatten()

torch.save(torch.from_numpy(train_labels), LABEL_TRAIN_FILE)

N_tr, N_va, N_te = len(train_labels), len(val_labels), len(test_labels)
C = int(train_labels.max()) + 1

In [ ]:
feats_tr = torch.load(os.path.join(DATA_PATH, FEAT_TRAIN_FILE),
                      map_location='cpu').numpy()
feats_va = torch.load(os.path.join(DATA_PATH, FEAT_VAL_FILE),
                      map_location='cpu').numpy()
feats_te = torch.load(os.path.join(DATA_PATH, FEAT_TEST_FILE),
                      map_location='cpu').numpy()

feats_all = np.vstack([feats_tr, feats_va, feats_te])
feats_norm_all = feats_all / (np.linalg.norm(feats_all, axis=1, keepdims=True) + EPS)
labels_all = np.concatenate([train_labels, val_labels, test_labels])
N_all, d = feats_norm_all.shape

nbrs = NearestNeighbors(n_neighbors=K+1).fit(feats_norm_all)
dists, _ = nbrs.kneighbors(feats_norm_all)
SIGMA2 = np.median(dists[:, -1] ** 2)
print(f"[DEBUG] SIGMA² = {SIGMA2:.3e}")

def build_knn_graph(Z, labels, K, sigma):
    N = Z.shape[0]
    nbrs = NearestNeighbors(n_neighbors=K+1).fit(Z)
    dists, inds = nbrs.kneighbors(Z)
    rows, cols, vals = [], [], []
    for i in range(N):
        for j in inds[i, 1:]:
            w = math.exp(-np.linalg.norm(Z[i] - Z[j])**2 / sigma**2)
            if labels[i] != labels[j]:
                w *= 0.5
            rows.append(i); cols.append(j); vals.append(w)
    W = csr_matrix((vals + vals, (rows + cols, cols + rows)), shape=(N, N))
    W = W + EPS * eye(N, N)
    print(f"[DEBUG] W nnz={W.nnz}, density={W.nnz/(N*N):.2e}")
    return W

A0_all = build_knn_graph(feats_norm_all, labels_all, K, math.sqrt(SIGMA2))
D0 = np.array(A0_all.sum(1)).ravel()
Dinv2_0 = csr_matrix((1.0/np.sqrt(D0+EPS), (np.arange(N_all), np.arange(N_all))),
                     shape=(N_all, N_all))
L_norm_all = eye(N_all, N_all) - Dinv2_0.dot(A0_all.dot(Dinv2_0))

P_all = inv(eye(N_all, N_all) - PROP_ALPHA * L_norm_all)
Z_prop_all = P_all.dot(feats_norm_all)
Z_prop_norm_all = Z_prop_all / (np.linalg.norm(Z_prop_all, axis=1, keepdims=True) + EPS)

W_prop = build_knn_graph(Z_prop_norm_all, labels_all, K, math.sqrt(SIGMA2))
deg_prop = np.array(W_prop.sum(1)).ravel()

feats_norm_tr = feats_norm_all[:N_tr]
feats_norm_va = feats_norm_all[N_tr:N_tr+N_va]
feats_norm_te = feats_norm_all[N_tr+N_va:]

Z_prop_tr     = Z_prop_norm_all[:N_tr]
Z_prop_va     = Z_prop_norm_all[N_tr:N_tr+N_va]
Z_prop_te     = Z_prop_norm_all[N_tr+N_va:]


def compute_label_propagation(W, labels, C, iters=2):
    N = W.shape[0]
    Dinv = csr_matrix((1.0/(np.array(W.sum(1)).ravel()+EPS),
                       (np.arange(N), np.arange(N))),
                      shape=(N, N))
    Y = np.eye(C, dtype=np.float32)[labels]
    for _ in range(iters):
        Y = Dinv.dot(W.dot(Y))
    ent = -np.sum(Y * np.log(Y + 1e-12), axis=1).mean()
    print(f"[DEBUG] Y_smooth entropy: {ent:.3f}")
    return Y

Y_smooth_all = compute_label_propagation(W_prop, labels_all, C)
Y_smooth_tr  = Y_smooth_all[:N_tr]

# class weights
cnts = Counter(train_labels)
tot  = sum(cnts.values())
class_weights = torch.tensor([tot/cnts[c] for c in range(C)],
                             device=device, dtype=torch.float32)

In [ ]:
def train_with_graph_reg(model, train_loader, val_loader,
                         Wg, degg, lam, cls_w, device,
                         optimizer, scheduler, scaler,
                         num_epochs, patience, checkpoint_path):
    best_val, pat_cnt = float('inf'), 0
    for ep in range(1, num_epochs+1):
        model.train()
        for imgs, y_soft, idxs in train_loader:
            imgs, y_soft = imgs.to(device), y_soft.to(device)
            idxs_np = idxs.numpy()
            optimizer.zero_grad()
            with torch.cuda.amp.autocast(enabled=bool(scaler)):
                out = model(imgs)

                h = -(y_soft * F.log_softmax(out, dim=1)).sum(1)
                sw = cls_w[y_soft.argmax(1)]
                loss_ce = (h * sw).mean()

                bd = torch.sqrt(torch.from_numpy(degg[idxs_np]+EPS).to(device)).unsqueeze(1)
                outn = out / bd
                Wb   = torch.from_numpy(Wg[idxs_np][:, idxs_np].toarray()).to(device)
                sqd  = (outn.unsqueeze(1) - outn.unsqueeze(0)).pow(2).sum(-1)
                loss_reg = 0.5 * (Wb * sqd).sum()
                loss = loss_ce + lam * loss_reg
            if scaler:
                scaler.scale(loss).backward()
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                scaler.step(optimizer)
                scaler.update()
            else:
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()

        model.eval()
        val_loss, corr, tot = 0., 0, 0
        with torch.no_grad():
            for imgs, yv, _ in val_loader:
                imgs, yv = imgs.to(device), yv.to(device)
                out = model(imgs)
                val_loss += -(yv * F.log_softmax(out,1)).sum(1).mean().item()
                preds = out.argmax(1)
                corr += (preds == yv.argmax(1)).sum().item()
                tot  += preds.size(0)
        val_loss /= len(val_loader)
        scheduler.step(val_loss)
        val_acc = corr / tot
        if val_loss < best_val:
            best_val, pat_cnt = val_loss, 0
            torch.save(model.state_dict(), checkpoint_path)
        else:
            pat_cnt += 1
            if pat_cnt >= patience:
                break
    return checkpoint_path

In [ ]:
res_val  = os.path.join(DATA_PATH, 'derma_vgg_3.csv')
res_test = os.path.join(DATA_PATH, 'derma_vgg_3.csv')
for pth, hdr in [(res_val,  ['percentage','mean_f1_macro','std_f1_macro',
                             'mean_train_time_s','std_train_time_s',
                             'mean_val_acc','std_val_acc',
                             'mean_hausdorff','std_hausdorff']),
                 (res_test,['percentage','mean_f1_macro','std_f1_macro',
                             'mean_train_time_s','std_train_time_s',
                             'mean_test_acc','std_test_acc',
                             'mean_hausdorff','std_hausdorff'])]:
    if os.path.exists(pth):
        os.remove(pth)
    with open(pth, 'w', newline='') as f:
        csv.writer(f).writerow(hdr)

val_ds    = NumpyDataset(os.path.join(DATA_PATH, 'val_images.npy'),
                         os.path.join(DATA_PATH, VAL_LABEL_FILE),
                         transform=val_transform)
test_ds   = NumpyDataset(os.path.join(DATA_PATH, 'test_images.npy'),
                         os.path.join(DATA_PATH, TEST_LABEL_FILE),
                         transform=val_transform)

val_loader  = DataLoader(SoftDataset(val_ds,   val_labels, C),
                         batch_size=BATCH, shuffle=False)
test_loader = DataLoader(SoftDataset(test_ds,  test_labels, C),
                         batch_size=BATCH, shuffle=False)

for p in PERCENTAGES:
    lam      = LAPLACE_LAMBDA * min(1.0, p/5.0)
    lr_s     = LR * min(1.0, p/5.0)

    metrics_v, metrics_t = [], []
    times, hd_vals, hd_tests = [], [], []

    for seed in [42, 43, 45]:
        random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
        if device.type == 'cuda': torch.cuda.manual_seed_all(seed)

        # coreset
        sel = np.load(os.path.join(DATA_PATH, f"{CORESET_PREF}{int(p)}.npy"))
        train_ds = NumpyDataset(
            os.path.join(DATA_PATH, 'train_images.npy'),
            os.path.join(DATA_PATH, 'train_labels.npy'),
            transform=train_transform
        )
        train_loader = DataLoader(
            CoresetDataset(train_ds, sel, Y_smooth_tr),
            batch_size=BATCH, shuffle=True
        )

        model = create_model('vgg16',
                             pretrained=True, num_classes=C).to(device)
        opt   = optim.Adam(model.parameters(), lr=lr_s)
        sch   = optim.lr_scheduler.ReduceLROnPlateau(opt, 'min', patience=PATIENCE)
        scaler= torch.cuda.amp.GradScaler() if device.type=='cuda' else None

        ckpt_path = os.path.join(DATA_PATH, f'best_p{p}_s{seed}.pth')
        t0 = time.time()
        best_ckpt = train_with_graph_reg(
            model, train_loader, val_loader,
            W_prop, deg_prop,
            lam, class_weights,
            device, opt, sch, scaler,
            EPOCHS, PATIENCE, ckpt_path
        )
        times.append(time.time() - t0)

        model.load_state_dict(torch.load(best_ckpt))
        preds_v, trues_v, idxs_v = [], [], []
        for imgs, yv, idxs in val_loader:
            out = model(imgs.to(device))
            preds_v.extend(out.argmax(1).cpu().tolist())
            trues_v.extend(yv.argmax(1).cpu().tolist())
            idxs_v.extend(idxs.tolist())
        metrics_v.append((f1_score(trues_v,preds_v,average='macro'),
                          accuracy_score(trues_v,preds_v)))

        X = feats_norm_va[idxs_v]
        Y = Z_prop_va[idxs_v]
        h_ab = directed_hausdorff(X, Y)[0]
        h_ba = directed_hausdorff(Y, X)[0]
        hd_vals.append(max(h_ab, h_ba))

        preds_t, trues_t, idxs_t = [], [], []
        for imgs, yv, idxs in test_loader:
            out = model(imgs.to(device))
            preds_t.extend(out.argmax(1).cpu().tolist())
            trues_t.extend(yv.argmax(1).cpu().tolist())
            idxs_t.extend(idxs.tolist())
        metrics_t.append((f1_score(trues_t,preds_t,average='macro'),
                          accuracy_score(trues_t,preds_t)))

        X = feats_norm_te[idxs_t]
        Y = Z_prop_te[idxs_t]
        h_ab = directed_hausdorff(X, Y)[0]
        h_ba = directed_hausdorff(Y, X)[0]
        hd_tests.append(max(h_ab, h_ba))

    arr_v = np.array(metrics_v)
    arr_t = np.array(metrics_t)

    row_v = [p,
             arr_v[:,0].mean(), arr_v[:,0].std(),
             np.mean(times),      np.std(times),
             arr_v[:,1].mean(),   arr_v[:,1].std(),
             np.mean(hd_vals),    np.std(hd_vals)]
    row_t = [p,
             arr_t[:,0].mean(), arr_t[:,0].std(),
             np.mean(times),      np.std(times),
             arr_t[:,1].mean(),   arr_t[:,1].std(),
             np.mean(hd_tests),   np.std(hd_tests)]

    with open(res_val,  'a', newline='') as f:
        csv.writer(f).writerow(row_v)
    with open(res_test, 'a', newline='') as f:
        csv.writer(f).writerow(row_t)